# After dark Transportation in Sydney

#### Introduction
RBT means you need a Plan B. This NSW road safety campaign slogan is clear and concise. However, it’s message on the perils of drink driving also highlights the key role an effective transportation system plays in the nightlight of a city.

With a wide and varying nightlife stretching across the city Sydney has many cafes, bars and clubs to offer. As the hub for international flights into Australia it receives over 4 million international visitors as well as 12 million domestic visitors a year.Clearly the transportation demand is there, and one of the crucial components of meeting that need is an efficient taxi and ride sharing economy.

A significant driver of the efficiency of this economy is the ability to pair providers with potential clients when and where the demand is. This report aims to identify the locations in Sydney where the demand for taxi's or ride sharing services are the greatest to assist in driver distribution.

#### Data
To solve the problem of allocating taxi and ride sharing serivces to the in demand locations in Sydney the following data has been used:

 * Wikipedia for list of Sydney Suburbs https://en.wikipedia.org/wiki/List_of_Sydney_suburbs

 * A file including all Australian Post Code location data from https://raw.githubusercontent.com/matthewproctor/australianpostcodes/master/australian_postcodes.csv
 
 * Foresquare data for cafe's, resturants, pubs and clubs in the Sydney area: establishment location, opening and closing hours, user ratings as a proxy for popularity and if the venue serves alcohol or not

#### Methodlogy
An initial analysis was conducted by scraping the list of Sydney suburbs from the 'List of Sydney Suburbs' Wikipedia page. This page along with the list of suburb names also provide a brief overview of the each suburbs ie. intercity or or family neighbourhood. These overviews then provide some indications of what suburbs to concentrate on.

After reveiwing the data provided and based on the restriction on API calls to foursquare it was decided to focus on the identified inner city suburbs of Syndey. These where chossen as the they are the heart of the city containing the Harbour precint, the Harbour bridge, the Rocks and the Opera house all key attractions for both international and domestic tourists. With a high concentration of bars, resturants and clubs they are also contain the bulk of the cities nightlife.

Once the initial review of the information was the Autralian Suburbs postcode and location data was retrive and entered into a data table. This data was then trimed down to the Sydney area Suburbs by locality. The data was then further  down whittled only Sydney city metro area suburs as identified previouly. Lastly the data was cleaned to remove duplicates, null entry rows and error rows. The final suburbs locations where then ploted on a map of Sydney to visualise the data.

In [34]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [35]:
df_pc = pd.read_csv('AustraliaPostCodes.csv')
df_pc.head() 

,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region
0,230,200,ANU,ACT,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R1
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,NaN,NaN,NaN,R1
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,Darwin City,701.0,Darwin,R1
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,Darwin City,701.0,Darwin,R1
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,Darwin Suburbs,701.0,Darwin,R1


In [36]:
list(df_pc.columns)

['id',
 'postcode',
 'locality',
 'state',
 'long',
 'lat',
 'dc',
 'type',
 'status',
 'sa3',
 'sa3name',
 'sa4',
 'sa4name',
 'region']

In [37]:
# Create data frame with only Sydney postcodes
df_pc = df_pc[df_pc['sa3name'].notna()]
df_pc = df_pc[df_pc['sa3name'].str.contains('Sydney')]
df_pc = df_pc[df_pc['dc'].notna()]
df_pc = df_pc[df_pc['dc'].str.contains('Sydney')] # 
df_pc = df_pc.drop_duplicates(subset='locality', keep="first") # remove duplicate locations
df_pc.head()

,id,postcode,locality,state,long,lat,dc,type,status,sa3,sa3name,sa4,sa4name,region
686,20208,2000,BARANGAROO,NSW,151.201580,-33.860520,Sydney Metro,Delivery Area,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
687,4478,2000,DARLING HARBOUR,NSW,151.256649,-33.859953,Sydney Metro,NaN,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
704,4494,2008,GOLDEN GROVE,NSW,151.193858,-33.891146,Sydney Metro,NaN,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
705,4495,2009,DARLING ISLAND,NSW,151.193055,-33.871222,Sydney Metro,NaN,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1
709,4499,2010,TAYLOR SQUARE,NSW,151.212262,-33.884119,Sydney Metro,NaN,Updated 6-Feb-2020,11703.0,Sydney Inner City,117.0,Sydney - City and Inner South,R1


In [38]:
# Clean Data to keep only applicable columns and reset index
df_pc = df_pc[['postcode', 'locality', 'long', 'lat']]
df_pc.reset_index(drop=True, inplace=True)
df_pc = df_pc.rename({'postcode': 'PostCode', 'locality': 'Suburb', 'long': 'Longitude',
                      'lat': 'Latitude'}, axis=1)  # new method
df_pc.head()

,PostCode,Suburb,Longitude,Latitude
0,2000,BARANGAROO,151.201580,-33.860520
1,2000,DARLING HARBOUR,151.256649,-33.859953
2,2008,GOLDEN GROVE,151.193858,-33.891146
3,2009,DARLING ISLAND,151.193055,-33.871222
4,2010,TAYLOR SQUARE,151.212262,-33.884119


In [39]:
df_pc.shape

(9, 4)

In [40]:
address = '100 Market St, Sydney, NSW 2000'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-33.8707038 151.2084686


In [41]:
# create map of Sydney using latitude and longitude values
map_sydney = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode, locality in zip(df_pc['Latitude'], df_pc['Longitude'], df_pc['PostCode'], df_pc['Suburb']):
    label = '{}, {}'.format(postcode, locality)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    
map_sydney

The foursquare API was then called to return the venue and venue locations in each suburb and the returned venues where the entered into a data table. A list of all the unique venue categories returns was then created. Analysing the list of venue categories the data table was then restricted to Resturants and Bars of various sorts. This was done as the goal of this report is to identify the key locations within Sydney City for transportation to facilitate the nightlife of the city. 

In [42]:
# Login to Foresquare
CLIENT_ID = 'GRD4D11PTH2NXCTBSBWYA2FWXDWS4H4VPH5ZDCY1EFSGQOPG' # Foursquare ID
CLIENT_SECRET = '244S2XZWSRYZW1DIZF44CEH0CRSYMVGG2YP3Z20OGCBMYXNB' # Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GRD4D11PTH2NXCTBSBWYA2FWXDWS4H4VPH5ZDCY1EFSGQOPG
CLIENT_SECRET:244S2XZWSRYZW1DIZF44CEH0CRSYMVGG2YP3Z20OGCBMYXNB


In [43]:
suburb_latitude = df_pc.loc[0, 'Latitude'] # suburb latitude value
suburb_longitude = df_pc.loc[0, 'Longitude'] # suburb longitude value

suburb_name = df_pc.loc[0, 'Suburb'] # suburb name

In [44]:
LIMIT = 100
radius = 500

In [45]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [46]:
def getVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    df_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    df_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue',
                  'Venue Id',
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(df_venues)

In [47]:
df_venues = getVenues(names=df_pc['Suburb'],
                                   latitudes=df_pc['Latitude'],
                                   longitudes=df_pc['Longitude']
                                  )
df_venues.groupby('Suburb').count()

BARANGAROO
DARLING HARBOUR
GOLDEN GROVE
DARLING ISLAND
TAYLOR SQUARE
KINGS CROSS
HMAS PLATYPUS
GORE HILL
KURRABA POINT


,Suburb Latitude,Suburb Longitude,Venue,Venue Id,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,,
BARANGAROO,25,25,25,25,25,25,25
DARLING HARBOUR,1,1,1,1,1,1,1
DARLING ISLAND,41,41,41,41,41,41,41
GOLDEN GROVE,32,32,32,32,32,32,32
GORE HILL,6,6,6,6,6,6,6
HMAS PLATYPUS,47,47,47,47,47,47,47
KINGS CROSS,100,100,100,100,100,100,100
KURRABA POINT,11,11,11,11,11,11,11
TAYLOR SQUARE,96,96,96,96,96,96,96


In [48]:
df_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Id,Venue Latitude,Venue Longitude,Venue Category
0,BARANGAROO,-33.86052,151.20158,The Langham Hotel Sydney,4b05875bf964a5203e8d22e3,-33.860517,151.203437,Hotel
1,BARANGAROO,-33.86052,151.20158,Palisade Hotel,4ca55895b7106dcb3b0e54a5,-33.857979,151.202264,Pub
2,BARANGAROO,-33.86052,151.20158,Lord Nelson Brewery Hotel,4b05875bf964a520558d22e3,-33.858403,151.203548,Brewery
3,BARANGAROO,-33.86052,151.20158,Fish at the Rocks,4b7905b5f964a52034e92ee3,-33.858673,151.203500,Seafood Restaurant
4,BARANGAROO,-33.86052,151.20158,Observatory Hill,4b5ebd9ff964a5204b9729e3,-33.859125,151.204977,Park


In [49]:
list(df_venues.columns)

['Suburb',
 'Suburb Latitude',
 'Suburb Longitude',
 'Venue',
 'Venue Id',
 'Venue Latitude',
 'Venue Longitude',
 'Venue Category']

In [50]:
df_venues['Venue Category'].unique()

array(['Hotel', 'Pub', 'Brewery', 'Seafood Restaurant', 'Park',
       'Planetarium', 'Café', 'Coffee Shop', 'Bakery', 'Hotel Bar',
       'Hostel', 'Donut Shop', 'Performing Arts Venue',
       'Australian Restaurant', 'Theater', 'Lebanese Restaurant',
       'Scenic Lookout', 'Turkish Restaurant', 'Italian Restaurant',
       'Harbor / Marina', 'Farmers Market', 'College Rec Center',
       'Thai Restaurant', 'Beer Garden', 'Market', 'Tea Room', 'Office',
       'Japanese Restaurant', 'Bar', 'Middle Eastern Restaurant',
       'Butcher', 'Fish Market', 'Breakfast Spot', 'Restaurant',
       'Ice Cream Shop', 'Gym', 'French Restaurant', 'Sushi Restaurant',
       'Asian Restaurant', 'Creperie', 'Malay Restaurant', 'Rock Club',
       'Indian Restaurant', 'Vietnamese Restaurant', 'Speakeasy',
       'Cocktail Bar', 'Yoga Studio', 'Vegetarian / Vegan Restaurant',
       'Ramen Restaurant', 'Pizza Place', 'Grocery Store',
       'Sandwich Place', 'Deli / Bodega', 'Argentinian Restaurant'

In [51]:
VList = ['Hotel', 'Pub', 'Seafood Restaurant', 'Hotel Bar', 'Australian Restaurant',
       'Theater', 'Lebanese Restaurant', 'Italian Restaurant', 'Thai Restaurant', 'Beer Garden',
       'Turkish Restaurant', 'Japanese Restaurant', 'Bar', 'Middle Eastern Restaurant',
       'Restaurant', 'French Restaurant', 'Sushi Restaurant', 'Asian Restaurant', 'Creperie',
       'Malay Restaurant', 'Rock Club', 'Indian Restaurant',
       'Vietnamese Restaurant', 'Speakeasy', 'Cocktail Bar',
       'Vegetarian / Vegan Restaurant', 'Ramen Restaurant',
       'Pizza Place', 'Argentinian Restaurant', 'Wine Bar', 'Greek Restaurant',
       'American Restaurant', 'Colombian Restaurant', 'Dessert Shop',
       'Fish & Chips Shop', 'Hawaiian Restaurant', 'Burger Joint',
       'Karaoke Bar', 'Korean Restaurant','Fast Food Restaurant', 'Moroccan Restaurant',
       'Dumpling Restaurant'] # Reduce list to 'nightlife' venues

In [52]:
df_venues = df_venues.loc[df_venues['Venue Category'].isin(VList)]
df_venues.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Id,Venue Latitude,Venue Longitude,Venue Category
0,BARANGAROO,-33.86052,151.20158,The Langham Hotel Sydney,4b05875bf964a5203e8d22e3,-33.860517,151.203437,Hotel
1,BARANGAROO,-33.86052,151.20158,Palisade Hotel,4ca55895b7106dcb3b0e54a5,-33.857979,151.202264,Pub
3,BARANGAROO,-33.86052,151.20158,Fish at the Rocks,4b7905b5f964a52034e92ee3,-33.858673,151.203500,Seafood Restaurant
10,BARANGAROO,-33.86052,151.20158,Blu Bar On 36,4b058760f964a520948e22e3,-33.861067,151.206361,Hotel Bar
13,BARANGAROO,-33.86052,151.20158,Shangri-La Hotel,4b05875df964a520c08d22e3,-33.861141,151.206460,Hotel


The locations of these venues where then ploted on the Sydney map to visulise any patterns. It was noted that the venues appear to cluster in six specific groupings.

In [53]:
#Create map of venues
map_venues = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue in zip(df_venues['Venue Latitude'], df_venues['Venue Longitude'], df_venues['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  
    
map_venues

In [54]:
Bars = ['Hotel', 'Pub', 'Hotel Bar', 'Bar', 'Rock Club', 'Speakeasy', 'Cocktail Bar',
       'Wine Bar', 'Karaoke Bar']
df_Bars = df_venues.loc[df_venues['Venue Category'].isin(Bars)]
df_Bars.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Id,Venue Latitude,Venue Longitude,Venue Category
0,BARANGAROO,-33.86052,151.20158,The Langham Hotel Sydney,4b05875bf964a5203e8d22e3,-33.860517,151.203437,Hotel
1,BARANGAROO,-33.86052,151.20158,Palisade Hotel,4ca55895b7106dcb3b0e54a5,-33.857979,151.202264,Pub
10,BARANGAROO,-33.86052,151.20158,Blu Bar On 36,4b058760f964a520948e22e3,-33.861067,151.206361,Hotel Bar
13,BARANGAROO,-33.86052,151.20158,Shangri-La Hotel,4b05875df964a520c08d22e3,-33.861141,151.206460,Hotel
15,BARANGAROO,-33.86052,151.20158,Harts Pub,4ba09757f964a520847337e3,-33.861870,151.206314,Pub


In [55]:
df_Food = df_venues.loc[df_venues['Venue Category'].isin(Bars) == False]
df_Food.head()

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Id,Venue Latitude,Venue Longitude,Venue Category
3,BARANGAROO,-33.86052,151.20158,Fish at the Rocks,4b7905b5f964a52034e92ee3,-33.858673,151.203500,Seafood Restaurant
16,BARANGAROO,-33.86052,151.20158,NOLA Smokehouse & Bar,58959f191ac004091d76bfca,-33.863840,151.201800,Australian Restaurant
17,BARANGAROO,-33.86052,151.20158,Roslyn Packer Theatre,5326b5f7498e83faf00916c0,-33.857019,151.204947,Theater
18,BARANGAROO,-33.86052,151.20158,El-Phoenician,4ed4a11002d5ef93a2401ca1,-33.856769,151.203286,Lebanese Restaurant
20,BARANGAROO,-33.86052,151.20158,Anason,56ccfdbacd10ade46ab9ff03,-33.864590,151.201310,Turkish Restaurant


For further analysis the data set was split into Bars and Food venues to see if there was any distinction in the groupings. These two groups where then ploted on the same Syney map to visulise with Food venues marked in gray and Bars in red. It can be noted that the grouping of both types are mostly overlappign and as such indicate that there is limited benefit from the seperation in anlysis.

In [ ]:
#Create map of venues
map_venues = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue in zip(df_Food['Venue Latitude'], df_Food['Venue Longitude'], df_Food['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='grey',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  

for lat, lng, venue in zip(df_Bars['Venue Latitude'], df_Bars['Venue Longitude'], df_Bars['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)
    
map_venues

Having identified the grouping visually a kmeans algorithm was employed to identify the optimal grouping of the clusters and idetify the centeroids and hence best place to for transportation pickup/waiting areas.

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

x1 = df_venues['Venue Latitude'].values
x2 = df_venues['Venue Longitude'].values

# create new plot and data
plt.plot()
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2)
colors = ['blue', 'green', 'lime', 'yellow', 'lightcoral', 'purple', 'cyan']

# KMeans algorithm 
K = 7

kmeans_model = KMeans(n_clusters=K).fit(X)

print(kmeans_model.cluster_centers_)
centers = np.array(kmeans_model.cluster_centers_)

plt.plot()
plt.title('k means centroids')

for i, l in enumerate(kmeans_model.labels_):
    plt.plot(x1[i], x2[i], color=colors[l], marker="o",ls='None')


plt.scatter(centers[:,0], centers[:,1], marker="x", color='r')
plt.show()

Using the kmeans algorithm seven optimal groupings where identified and the coordinates for the centeroid captured in a data table.

In [ ]:
df_centers = pd.DataFrame(centers, columns=['lat', 'long'])
df_centers

In [ ]:
#Create map of venues
map_venues = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue in zip(df_venues['Venue Latitude'], df_venues['Venue Longitude'], df_venues['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_venues)  
    
# add centers to map to show best location for taxi/ride sharing rank
for lat, lng in zip(df_centers['lat'], df_centers['long']):
    label = 'Center'
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        icon=folium.Icon(color='lightgray',
                         icon='thumbs-up')).add_to(map_venues)     
    
map_venues

Lastly the centeroids where ploted on the map of Sydney with the venues included to visualise the locations.

#### Results
As a result of the analysis seven clusters of bars and resturants within the Sydney metropolitian area have been identified. Base on the the clustering of the venues at these locations the center or optimal point for taxi ranks or ride sharing services to wait has been identified.

In [ ]:
df_centers

#### Discussion
The results above show the optimal areas for ride sharing services or taxis to congregate in order to take passengers home after enjoing the Syndey nightlife.

However these recommendations do not take into account the proximty of other forms of public transportation such as busses or trains which maybe used as alternatives. Although from anecdotal evidence people are much less likely to use public transportation after a 'night out' which indicates that this may not be a factor in location although this hypothesis would have to be further substantiated.

Further this data is based on on location and type of venue. There is likely also to be a time factor in indentifying the of optimal transportation locations with variations due to the time of day. This would be something to investigate further to see if locations change due to differing venue closing times or day of the week and popularity of locations ie bars may have later times compared to family orientated restaurants.

#### Conclusion
In conclusion this report provides the optimal locations for transportation pickup point with in the Sydney city area to provide services for locals and tourists enjoying the local nightlife.

It is recommended that these location are noted by taxi and ride sharing companies so that the can maximise the efficiency of their service as well as ensuring the safe enjoyment of the patrons of the venues.